Loading the required packages. eppy has the required energyplus functions

In [ ]:
from eppy import modeleditor
from eppy.modeleditor import IDF
import pandas as pd
import matplotlib.pyplot as plt

Download the energyplus 9.5.0 version and then set the oath to the idd file (located in your local computer)

In [ ]:
iddfile = r'EnergyPlusV9-5-0/Energy+.idd' #your path to the IDD file: normally inside your directory where Energy PLus is
IDF.setiddname(iddfile)

This is a function to parse the date time in the output file

In [ ]:
def parse_energyplus_datetime_string(st, year=2021):
    st=st.strip()
    month=int(st[0:2])
    day=int(st[3:5])
    hour=int(st[7:9])
    minute=int(st[10:12])
    second=(st[13:15])
    if not hour==24:
        dt=pd.Timestamp(year,month,day,hour,minute)
    else:
        hour=0
        dt=pd.Timestamp(year,month,day,hour,minute)
        dt+=pd.Timedelta('1 day')
    return dt

In [ ]:
These lines of code run the prametric simulations using EnergyPlus. You can decide the number of simulations N and the paramet

In [ ]:
base = r'PVHPSFD.idf' 
idf1 = IDF(base)

windows = idf1.idfobjects["WindowMaterial:SimpleGlazingSystem"]
walls = idf1.idfobjects["Material"]
people = idf1.idfobjects["People"]
lights = idf1.idfobjects["Lights"]
electricequip = idf1.idfobjects["ElectricEquipment"]
gasequip = idf1.idfobjects["GasEquipment"]
dhw = idf1.idfobjects["WaterHeater:Mixed"]
sh = idf1.idfobjects["Coil:Heating:Fuel"]

from pyDOE import lhs
N = 1 #Number of simulations
seq = lhs(1, N) # this creates a sequence; the first number is the number of dimensions and the second number is the number of simulations or samples

cond = []   #for conductivity
elec = pd.DataFrame()   #for electricity consumption
gas = pd.DataFrame()    #for heat consumption
heat = pd.DataFrame() 
dhw = pd.DataFrame() 
pvproduction = pd.DataFrame() 
    
for i in range(N):    #Second loop to create randomized sequences: this runs for the length of number of samples
    for SimpleGlazingSystem in windows:
        SimpleGlazingSystem.UFactor = 0.5 + seq[i][0]*(3-2.5)
    for Conductivity in walls:
        walls[11].Conductivity = 0.03186 + seq[i][0]*(0.099-0.080)
        walls[15].Conductivity = 0.0365 + seq[i][0]*(0.099-0.080)
        walls[16].Conductivity = 0.06009 + seq[i][0]*(0.099-0.080)
        walls[17].Conductivity = 0.0463 + seq[i][0]*(0.099-0.080)
        walls[21].Conductivity = 0.04577 + seq[i][0]*(0.090-0.080)
    for Number_of_People in people:
        people[0].Number_of_People = 2 + seq[i][0]*(4-2)
    for Mixed in dhw:
        dhw.Heater_Thermal_Efficiency = 0.9 + seq[i][0]*(0.80-0.70)
    for Heating in sh:
        sh.Burner_Efficiency = 0.9 + seq[i][0]*(0.80-0.70)
    for Design_Level in gasequip:
        gasequip[0].Design_Level = 250 + seq[i][0]*(400-300)
        gasequip[1].Design_Level = 350 + seq[i][0]*(400-500)            
    for Design_Level in electricequip:
        electricequip[0].Design_Level = 40 + seq[i][0]*(80-50)
        electricequip[1].Design_Level = 40 + seq[i][0]*(100-80)
        electricequip[2].Design_Level = 15 + seq[i][0]*(30-20)
        electricequip[3].Design_Level = 200 + seq[i][0]*(25-15)
        electricequip[4].Design_Level = 200 + seq[i][0]*(60-40)
        electricequip[5].Design_Level = 20 + seq[i][0]*(60-40)
    for Watts_per_Zone_Floor_Area in lights:
        lights[0].Watts_per_Zone_Floor_Area = 0.70 + seq[i][0]*(1.5-1.3)
        lights[1].Watts_per_Zone_Floor_Area = 0.13 + seq[i][0]*(0.99-0.80)
    idf1.save()  #Save your modified IDF file| 
        
        
    idfname = r'PVHPSFD.idf' #"path to the new idf"
    epwfile = r'sim\weatherfile\3.epw'
    od = r'sim\od'
    idf = IDF(idfname, epwfile)
        
    idf.run(output_directory=od, readvars=True) #Run the simulation 
        
        #Extract and store the variables you need
    cond.append(SimpleGlazingSystem.UFactor)
        
       
    df=pd.read_csv(r'sim\od\eplusout.csv', 
               parse_dates=[0],
               index_col=[0],
               date_parser=parse_energyplus_datetime_string 
              ) # the html file you want to read
        
    dfelec = df.iloc[:, [3]]
    dfgas = df.iloc[:, [4]]
    dfheat = df.iloc[:, [7]]
    dfdhw = df.iloc[:, [8]]
    dfpvprod = df.iloc[:, [9]]
        
    elec = pd.concat([elec, dfelec], axis = 1)
    gas = pd.concat([gas, dfgas], axis = 1)
    heat = pd.concat([heat, dfheat], axis = 1)
    dhw = pd.concat([dhw, dfdhw], axis = 1)
    #pvproduction = pd.concat([pvproduction, dfpvprod], axis = 1)